# Medium-size system: Analysis

Here we collect the results of the sweep simulation with 2000 particles.

### Setup

In [ ]:
import pathlib

import numpy as np
import pandas as pd
from mpl_toolkits import mplot3d
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
import seaborn as sns

import lennardjonesium as lj

In [ ]:
chunk_count = 1
chunk_index = 0

sweep_config_file = pathlib.Path('sweep.ini')
sweep_config = lj.SweepConfiguration.from_file(sweep_config_file)
sweep_result = lj.orchestration.SweepResult(sweep_config_file, chunk_count, chunk_index)

In [ ]:
observations = pd.DataFrame()
for simulation_dir in sweep_config.simulation_dir_range(chunk_count, chunk_index):
    run_config = lj.Configuration.from_file(simulation_dir / sweep_config.templates.run_config_file)
    run_observations = pd.read_csv(
        simulation_dir / run_config.filepaths.observation_log).drop(columns=['TimeStep'])
    observations = pd.concat([observations, run_observations])

## Observations

### Energy landscape

In [ ]:
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(projection='3d')


ax.scatter3D(
    observations['Density'],
    observations['Temperature'],
    observations['TotalEnergy'],
    c=observations['TotalEnergy'],
    cmap='hsv',
    s=10
)

ax.set_xlabel('Density')
ax.set_ylabel('Temperature')
ax.set_zlabel('Energy')
ax.azim = -70
ax.elev = 10

plt.title("Energy vs Density, Temperature")
plt.show()

We can see a bit more detail, thanks to the greater resolution (30x30 vs 20x27), as well as the greater precision one gets when using a larger number of particles.  There appears to be a distinct, sharp cliff running from density 0.6 at low temperatures, up through density 1.0 at high temperatures.  It appears that there may also be a curved second-order phase transition line that meets this cliff in a triple point near density 0.8, temperature 0.5.

In [ ]:
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot()

ax.tripcolor(
    observations['Density'],
    observations['Temperature'],
    observations['TotalEnergy'],
    cmap='Spectral_r'
)

ax.set_xlabel('Density')
ax.set_ylabel('Temperature')

plt.title("Energy vs Density, Temperature")
plt.show()

The colormap version of the plot shows these features a bit more clearly.  It appears there may also be a horizontal cliff in the lower left, for densities less than 0.7 and temperature around 0.3.

### Pressure

In [ ]:
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot()

ax.tripcolor(
    observations['Density'],
    observations['Temperature'],
    observations['Pressure'],
    cmap='Spectral_r'
)

ax.set_xlabel('Density')
ax.set_ylabel('Temperature')

plt.title("Pressure vs Density, Temperature")
plt.show()

We observed in the Small System simulation that there may be some interesting inflection point around density 1.0.  Here we can see it in more detail, although it is still a bit unclear what exactly is going on.

### Diffusivity

In [ ]:
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot()

ax.tripcolor(
    observations['Density'],
    observations['Temperature'],
    observations['DiffusionCoefficient'],
    cmap='Spectral_r'
)

ax.set_xlabel('Density')
ax.set_ylabel('Temperature')

plt.title("Diffusivity vs Density, Temperature")
plt.show()

The diffusivity plot is still a bit difficult to make out.  It appears there is a lot of noise in this particular measurement.  We would like to see features that correspond in some way to the structure we have observed in the energy plot.

## Phase transitions via final state visualization

Next we will investigate a few specific points in the phase diagram to learn what phase is likely there.

Density 0.6 looks interesting.  Let's choose some points at this density and look above and below the conjectured phase transition line to see if there is anything interesting here.

### Density 0.6, temperature 1.0

In [ ]:
snapshot_data_d06_T10 = pd.read_csv('data/T_1.017241/d_0.593103/snapshots.csv', header=[0,1])
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(projection='3d')

ax.scatter3D(
    snapshot_data_d06_T10[('Position', 'X')],
    snapshot_data_d06_T10[('Position', 'Y')],
    snapshot_data_d06_T10[('Position', 'Z')],
    c=snapshot_data_d06_T10[('Position', 'Z')],
    cmap='hsv',
    s=7
)

plt.title("Particles at d=0.6, T=1.0")
plt.show()

Above we see the final positions of the particles for density 0.6 and temperature 1.0.  The particles look quite randomly distributed, so we conclude that this high-temperature phase is a fluid, as we might expect.

### Density 0.6, temperature 0.36

In [ ]:
snapshot_data_d06_T036 = pd.read_csv('data/T_0.362069/d_0.593103/snapshots.csv', header=[0,1])
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(projection='3d')

ax.scatter3D(
    snapshot_data_d06_T036[('Position', 'X')],
    snapshot_data_d06_T036[('Position', 'Y')],
    snapshot_data_d06_T036[('Position', 'Z')],
    c=snapshot_data_d06_T036[('Position', 'Z')],
    cmap='hsv',
    s=7
)

plt.title("Particles at d=0.6, T=0.36")
plt.show()

Here we see the final particle positions for density 0.6 and temperature 0.36.  This puts us slightly the horizontal "shelf" we have observed in the energy plot.  The particles appear randomly distributed, but there are holes, so they do not fill all of the available space.  From this picture alone, I would conclude we are in a liquid state, but it would be wise to check the mean square displacement.

In [ ]:
thermoynamic_data_d06_T036 = pd.read_csv('data/T_0.362069/d_0.593103/thermodynamics.csv')

plt.figure(figsize=(16,6))
plt.title("Mean Square Displacement vs time")
sns.lineplot(data=thermoynamic_data_d06_T036['MeanSquareDisplacement'])

plt.title("Mean Square Displacement vs Time at d=0.6, T=0.36")
plt.show()

The mean square displacement is certainly increasing linearly, so I believe the "liquid" conclusion is correct.

### Density 0.6, temperature 0.17

In [ ]:
snapshot_data_fluid = pd.read_csv('data/T_0.165517/d_0.593103/snapshots.csv', header=[0,1])
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(projection='3d')

ax.scatter3D(
    snapshot_data_fluid[('Position', 'X')],
    snapshot_data_fluid[('Position', 'Y')],
    snapshot_data_fluid[('Position', 'Z')],
    c=snapshot_data_fluid[('Position', 'Z')],
    cmap='hsv',
    s=7
)

plt.title("Particles at d=0.6, T=0.17")
plt.show()

Here we see the final particle positions for density 0.6 and temperature 0.17.  This puts *below* the horizontal "shelf" we have observed in the energy plot.  The particles are clearly arranged in a crystalline structure, so this is a solid state.  Therefore we conclude that the "shelf" line is a freezing/melting phase transition.